In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector
from keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt

In [2]:
n_in = 1000 #历史数量
n_out = 60 #预测数量
n_features = 1
# n_test = 1
n_val = 1
n_epochs = 250

In [3]:
#d导入数据
def load_stw_data() -> pd.DataFrame:
    
    df_stw = pd.read_excel('heartrate.xls')
    df_stw.columns = ['Time', 'Heartrate']
    
    return df_stw

In [4]:
#MinMaxScaler数据归一化，可以帮助网络模型更快的拟合，稍微有一些提高准确率的效果
def minmaxscaler(data: pd.DataFrame) -> pd.DataFrame:
    
    volume = data.Heartrate.values
    volume = volume.reshape(len(volume), 1)
    volume = scaler.fit_transform(volume)
    volume = volume.reshape(len(volume),)    
    data['Heartrate'] = volume
        
    return data

In [5]:
#划分训练数据集和验证数据集,这里需要注意的是我么需要预测的数据是不可以出现在训练中的，切记。
def split_data(x, y, n_test: int):
    
    x_train = x[:-n_val-n_out+1]
    x_val = x[-n_val:]
    y_train = y[:-n_val-n_out+1]
    y_val = y[-n_val:]
    
    return x_train, y_train, x_val, y_val

In [6]:
#划分X和Y
def build_train(train, n_in, n_out):
    
    train = train.drop(["Time"], axis=1)
    X_train, Y_train = [], []
    for i in range(train.shape[0]-n_in-n_out+1):
        X_train.append(np.array(train.iloc[i:i+n_in]))
        Y_train.append(np.array(train.iloc[i+n_in:i+n_in+n_out]["Heartrate"]))
        
    return np.array(X_train), np.array(Y_train)

In [7]:
#构建最简单的LSTM
def build_lstm(n_in: int, n_features: int):
    
    model = Sequential()
    model.add(LSTM(12, activation='relu', input_shape=(n_in, n_features)))
    model.add(Dropout(0.3))
    model.add(Dense(n_out))
    model.compile(optimizer='adam', loss='mae')
    
    return model

In [8]:
#模型拟合
def model_fit(x_train, y_train, x_val, y_val, n_features):
    
    model = build_lstm(
        n_in   = n_in,
        n_features= 1
    )
    model.compile(loss='mae', optimizer='adam')
    model.fit(x_train, y_train, epochs=n_epochs, batch_size=128, verbose=1,  validation_data=(x_val, y_val))
    m = model.evaluate(x_val, y_val)
    print(m)
    
    return model

In [9]:
data = load_stw_data()
scaler = MinMaxScaler(feature_range=(0, 1))
data = minmaxscaler(data)

In [10]:
data_copy = data.copy()
x, y = build_train(data_copy, n_in, n_out)
x_train, y_train, x_val, y_val = split_data(x, y, n_val)
model = build_lstm(n_in, 1)
model = model_fit(x_train, y_train, x_val, y_val, 1)
predict = model.predict(x_val)  

Epoch 1/250
38/38 [==============================] - 14s 348ms/step - loss: 0.4059 - val_loss: 0.4075
Epoch 2/250
24/38 [=================>............] - ETA: 4s - loss: nan

KeyboardInterrupt: 

In [ ]:
predict

array([[0.44542187, 0.44186002, 0.44346562, 0.44307303, 0.44366744,
        0.44508046, 0.45011416, 0.45777977, 0.45519578, 0.451154  ,
        0.4454422 , 0.45003313, 0.44656998, 0.44736886, 0.44666624,
        0.45032918, 0.446486  , 0.44685566, 0.4471466 , 0.4471451 ,
        0.4458397 , 0.4460389 , 0.4505354 , 0.44934523]], dtype=float32)